In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
## Environment
# Change main system path to be able to run code from src folder
import sys
p = sys.path[0]
# Mac OS
if sys.path[0].endswith('/models'):
    main_path = p[:-len('/models')]
sys.path[0] = main_path

import os, gc
from termcolor import colored
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from src import (config, fe, features, feature_selection, 
                 preprocess, training)
from src.fe import CreateFeatures

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
# DATA PREPARATION
# ===== LOAD DATA ======
metadata = pd.read_csv(config.DATA_DIR + 'metadata.csv')
print(f'Metadata: {metadata.shape}')

train_labels = pd.read_csv(config.DATA_DIR + 'train_labels.csv')
print(f'Train labels: {train_labels.shape}')

valid_labels = pd.read_csv(config.DATA_DIR + 'val_labels.csv')
print(f'Train labels: {valid_labels.shape}')

# Combine train and valid labels
trvl_labels = pd.concat([train_labels, valid_labels], axis = 0)

submission = pd.read_csv(config.DATA_DIR + 'submission_format.csv')
print(f'Submission: {submission.shape}')

# ===== FILE PATHS OF SAMPLES =====
train_files = metadata[metadata.split == 'train']['features_path'].to_dict()
valid_files = metadata[metadata.split == 'val']['features_path'].to_dict()
test_files = metadata[metadata.split == 'test']['features_path'].to_dict()
# Train & Valid files
trva_files = train_files.copy()
trva_files.update(valid_files)
# All files
all_test_files = valid_files.copy()
all_test_files.update(test_files)

# Define SAM testbed files
sam_files = metadata[(metadata.instrument_type == 'sam_testbed') & (metadata.split == 'train')]['features_path']
sam_files = sam_files.to_dict()

# Get the names of the target columns in a list
target_labels_list = [i for i in train_labels.columns if i not in ['sample_id']]
print(target_labels_list)

# SAM testbed labels
sam_labels = train_labels.drop(train_labels.tail(len(sam_files)).index)
sam_labels = pd.concat([sam_labels, valid_labels], axis=0)
print(f'Labels w/o SAM : {sam_labels.shape}')

Metadata: (1570, 5)
Train labels: (766, 11)
Train labels: (293, 11)
Submission: (804, 11)
['basalt', 'carbonate', 'chloride', 'iron_oxide', 'oxalate', 'oxychlorine', 'phyllosilicate', 'silicate', 'sulfate', 'sulfide']
Labels w/o SAM : (1047, 11)


In [4]:
FTS_NAME = 'fts_range_abun_to_temp'                 # Name of the file with base features for TRAINING
COMPUTE_FTS = False                         # Should the features be recomputed
COMPUTE_FTS_SAM = False                      # Compute SAM test bed
MODEL_ALGO = 'XGB_opt'                      # Name of the classifier
MODEL_NAME = FTS_NAME + '_' + MODEL_ALGO    # Name of the model
COMBINE_FTS = None                          # Feature sets to combine for training
NEW_FEATURES = None                         # Name of a data frame with new features to add to model
TRAIN_FTS_SFM = None                        # Features selected with SMF() for training.
BASE_MODEL = None

**COMPUTE FEATURES**

- Change the `fe._` method depending on the feature that we wish to calculate

In [5]:
# Check if feature is computed and load it or choose to compute it
check_file = 0
for i in ['_tr', '_trvl', '_vlte']:
    check_file += os.path.exists(os.path.join(config.DATA_DIR_OUT, FTS_NAME +
                                              str(i) + '.csv'))

if (check_file == 3) & (not COMPUTE_FTS):
    print('Reading features ... ')
    X_tr = pd.read_csv(os.path.join(config.DATA_DIR_OUT, FTS_NAME + '_tr.csv'))
    print(X_tr.shape)
    X_trvl = pd.read_csv(os.path.join(config.DATA_DIR_OUT, FTS_NAME + '_trvl.csv'))
    print(X_trvl.shape)
    X_vlte = pd.read_csv(os.path.join(config.DATA_DIR_OUT, FTS_NAME + '_vlte.csv'))
    print(X_vlte.shape)
    
else:
    print('Computing features ... ')
    # ----- TRAIN -----
    fe = CreateFeatures(metadata, train_files, 'tr', FTS_NAME)
    X_tr = fe.fts_topmz()
    print(colored(f'train => {X_tr.shape}', 'blue'))
    
    # ----- TRAIN & VALID -----
    fe = CreateFeatures(metadata, trva_files, 'trvl', FTS_NAME)
    X_trvl = fe.fts_topmz()
    print(colored(f'train => {X_trvl.shape}', 'blue'))
    
    # ----- VALID & TEST -----
    fe = CreateFeatures(metadata, all_test_files, 'vlte', FTS_NAME)
    X_vlte = fe.fts_topmz()
    print(colored(f'train => {X_vlte.shape}', 'blue'))
    
if COMPUTE_FTS_SAM:
    print(f'\nCreating SAM testbed samples ...')
    # Training without SAM testbed
    X_tr_sam = X_tr.drop(X_tr.tail(len(sam_files)).index).copy()
    X_tr_sam = pd.concat([X_tr_sam, X_trvl.iloc[len(train_files):,:]], axis=0)
    print(f'Train shape: {X_tr_sam.shape}')
    # Validation data
    X_vl_sam = X_tr.tail(len(sam_files)).copy()
    print(f'Valid shape: {X_vl_sam.shape}')

Reading features ... 
(766, 1583)
(1059, 1583)
(804, 1583)


## MODELS

**TRAIN**

In [6]:
# ===== TRAIN =====
split_type = 'tr'

# Initialize the feature selection class
smf = feature_selection.SelectModelFeatures(
    base_sfm_features_name=TRAIN_FTS_SFM,
    base_fitted_model_name=BASE_MODEL,
    target_labels_list=target_labels_list,
    new_features_file_name=NEW_FEATURES,
    fitted_model_name=MODEL_NAME,
    fitted_model_algo=MODEL_ALGO,
    X_tr=X_tr,
    X_vlte=X_vlte,
    split_type='tr',
    train_labels=train_labels,
    valid_files=valid_files,
    valid_labels=valid_labels)

if TRAIN_FTS_SFM:
    # Loads FTS_NAME_tr_SFM_COLS - cols to train with if
    # training is done without full column lenght of input data
    TRAIN_FTS_DICT = smf.load_features(file_name=FTS_NAME)
else: 
    TRAIN_FTS_DICT = None
    
# Train the model- saves features as MODEL_NAME_tr_COLS.txt'
cvloss, submission_model = training.train_tbl(
    df_train=X_tr,
    df_labels=train_labels,
    target_list=target_labels_list,
    df_test=X_vlte,
    split_type=split_type,
    model_algo=MODEL_ALGO,
    sub_name=MODEL_NAME + '_' + split_type,
    fts_select_cols=TRAIN_FTS_DICT,
    )

# Compute validation loss when full model is trained
mloss, mloss_avg = training.compute_valid_loss(submission_model,
                                               valid_files, valid_labels,
                                               target_labels_list,
                                               sub_name=MODEL_NAME + '_' + split_type,
                                               fts_select_cols=TRAIN_FTS_DICT)
print(colored(f'VAL LogLoss: {np.round(mloss_avg, 5)}', 'green'))

CV training ....
basalt
LogLoss 0.20889998844896987
carbonate
LogLoss 0.1345570790091733
chloride
LogLoss 0.21107801545434884
iron_oxide
LogLoss 0.2579356809837416
oxalate
LogLoss 0.024436258618646945
oxychlorine
LogLoss 0.18790611290852416
phyllosilicate
LogLoss 0.27847031184215276
silicate
LogLoss 0.25446846937888007
sulfate
LogLoss 0.24318406384552899
sulfide
LogLoss 0.07449835172341078
Full training .....
basalt - nfeatures: 1583
carbonate - nfeatures: 1583
chloride - nfeatures: 1583
iron_oxide - nfeatures: 1583
oxalate - nfeatures: 1583
oxychlorine - nfeatures: 1583
phyllosilicate - nfeatures: 1583
silicate - nfeatures: 1583
sulfate - nfeatures: 1583
sulfide - nfeatures: 1583
Saving fts_range_abun_to_temp_XGB_opt_tr_COLS.txt
CV LogLoss: 0.18754
VAL LogLoss: 0.47154


In [7]:
# Compute new features selected from the fitted model
# If just to read the features use compute_features=False
# Saves features as Loads FTS_NAME_tr_SFM_COLS
SFM_COLUMNS = smf.select_features(cv_new_model=cvloss)
smf.show_no_fts_label(SFM_COLUMNS)

Recomputing current model features from fitted model fts_range_abun_to_temp_XGB_opt_tr
Computing optimal threshold for each label
basalt - Computing threshold on fts_range_abun_to_temp_XGB_opt_tr_basalt.joblib.dat
carbonate - Computing threshold on fts_range_abun_to_temp_XGB_opt_tr_carbonate.joblib.dat
chloride - Computing threshold on fts_range_abun_to_temp_XGB_opt_tr_chloride.joblib.dat
iron_oxide - Computing threshold on fts_range_abun_to_temp_XGB_opt_tr_iron_oxide.joblib.dat
oxalate - Computing threshold on fts_range_abun_to_temp_XGB_opt_tr_oxalate.joblib.dat
oxychlorine - Computing threshold on fts_range_abun_to_temp_XGB_opt_tr_oxychlorine.joblib.dat
phyllosilicate - Computing threshold on fts_range_abun_to_temp_XGB_opt_tr_phyllosilicate.joblib.dat
silicate - Computing threshold on fts_range_abun_to_temp_XGB_opt_tr_silicate.joblib.dat
sulfate - Computing threshold on fts_range_abun_to_temp_XGB_opt_tr_sulfate.joblib.dat
sulfide - Computing threshold on fts_range_abun_to_temp_XGB_op

In [8]:
# Retrain the model with newly selected features
TRAIN_FTS_SFM = 'fts_range_abun_to_temp'            # Features selected with SMF() for training.


# ===== TRAIN =====
split_type = 'tr'

if TRAIN_FTS_SFM:
    # Loads FTS_NAME_tr_SFM_COLS - cols to train with if
    # training is done without full column lenght of input data
    TRAIN_FTS_DICT = smf.load_features(file_name=TRAIN_FTS_SFM)
else: 
    TRAIN_FTS_DICT = None
    
# Train the model- saves features as MODEL_NAME_tr_COLS.txt'
cvloss, submission_model = training.train_tbl(
    df_train=X_tr,
    df_labels=train_labels,
    target_list=target_labels_list,
    df_test=X_vlte,
    split_type=split_type,
    model_algo=MODEL_ALGO,
    sub_name=MODEL_NAME + '_' + split_type,
    fts_select_cols=TRAIN_FTS_DICT,
    )

# Compute validation loss when full model is trained
mloss, mloss_avg = training.compute_valid_loss(submission_model,
                                               valid_files, valid_labels,
                                               target_labels_list,
                                               sub_name=MODEL_NAME + '_' + split_type,
                                               fts_select_cols=TRAIN_FTS_DICT)
print(colored(f'VAL LogLoss: {np.round(mloss_avg, 5)}', 'green'))

Loading feature column names
Reading fts_range_abun_to_temp_XGB_opt_tr_SFM_COLS.txt
CV training ....
basalt
LogLoss 0.20889998844896987
carbonate
LogLoss 0.12862492236569653
chloride
LogLoss 0.18530064695767184
iron_oxide
LogLoss 0.23376275089563162
oxalate
LogLoss 0.020860387664823413
oxychlorine
LogLoss 0.18790611290852416
phyllosilicate
LogLoss 0.27847031184215276
silicate
LogLoss 0.20407440353327116
sulfate
LogLoss 0.22213785688095258
sulfide
LogLoss 0.09315464450122711
Full training .....
basalt - nfeatures: 1583
carbonate - nfeatures: 101
chloride - nfeatures: 154
iron_oxide - nfeatures: 247
oxalate - nfeatures: 11
oxychlorine - nfeatures: 1583
phyllosilicate - nfeatures: 1583
silicate - nfeatures: 156
sulfate - nfeatures: 41
sulfide - nfeatures: 7
Saving fts_range_abun_to_temp_XGB_opt_tr_COLS_sfm.txt
CV LogLoss: 0.17632
VAL LogLoss: 0.16085


**TRAIN & VALID**

In [9]:
FTS_NAME = 'fts_range_abun_to_temp'                 # Name of the file with base features for TRAINING
COMPUTE_FTS = False                         # Should the features be recomputed
COMPUTE_FTS_SAM = False                      # Compute SAM test bed
MODEL_ALGO = 'XGB_opt'                      # Name of the classifier
MODEL_NAME = FTS_NAME + '_' + MODEL_ALGO    # Name of the model
COMBINE_FTS = None                          # Feature sets to combine for training
NEW_FEATURES = None                         # Name of a data frame with new features to add to model
TRAIN_FTS_SFM = None                        # Features selected with SMF() for training.
BASE_MODEL = None

In [10]:
# ===== TRAIN =====
split_type = 'trvl'

# Initialize the feature selection class
smf = feature_selection.SelectModelFeatures(
    base_sfm_features_name=TRAIN_FTS_SFM,
    base_fitted_model_name=BASE_MODEL,
    target_labels_list=target_labels_list,
    new_features_file_name=NEW_FEATURES,
    fitted_model_name=MODEL_NAME,
    fitted_model_algo=MODEL_ALGO,
    X_tr=X_trvl,
    X_vlte=X_vlte,
    split_type=split_type,
    train_labels=trvl_labels,
    valid_files=valid_files,
    valid_labels=valid_labels)

if TRAIN_FTS_SFM:
    # Loads FTS_NAME_tr_SFM_COLS - cols to train with if
    # training is done without full column lenght of input data
    TRAIN_FTS_DICT = smf.load_features()
else: 
    TRAIN_FTS_DICT = None
    
# Train the model- saves features as MODEL_NAME_tr_COLS.txt'
cvloss, submission_model = training.train_tbl(
    df_train=X_trvl,
    df_labels=trvl_labels,
    target_list=target_labels_list,
    df_test=X_vlte,
    split_type=split_type,
    model_algo=MODEL_ALGO,
    sub_name=MODEL_NAME + '_' + split_type,
    fts_select_cols=TRAIN_FTS_DICT,
    )

# Compute validation loss when full model is trained
mloss, mloss_avg = training.compute_valid_loss(submission_model,
                                               valid_files, valid_labels,
                                               target_labels_list,
                                               sub_name=MODEL_NAME + '_' + split_type,
                                               fts_select_cols=TRAIN_FTS_DICT)
print(colored(f'VAL LogLoss: {np.round(mloss_avg, 5)}', 'green'))

CV training ....
basalt
LogLoss 0.1648905728523552
carbonate
LogLoss 0.1043806934118106
chloride
LogLoss 0.18591879225640234
iron_oxide
LogLoss 0.21104580038718743
oxalate
LogLoss 0.013862204365233008
oxychlorine
LogLoss 0.1554832131014689
phyllosilicate
LogLoss 0.25760361716934876
silicate
LogLoss 0.22854694057843603
sulfate
LogLoss 0.18535595144352082
sulfide
LogLoss 0.08403620244280724
Full training .....
basalt - nfeatures: 1583
carbonate - nfeatures: 1583
chloride - nfeatures: 1583
iron_oxide - nfeatures: 1583
oxalate - nfeatures: 1583
oxychlorine - nfeatures: 1583
phyllosilicate - nfeatures: 1583
silicate - nfeatures: 1583
sulfate - nfeatures: 1583
sulfide - nfeatures: 1583
Saving fts_range_abun_to_temp_XGB_opt_trvl_COLS.txt
CV LogLoss: 0.15911
VAL LogLoss: 0.38433


In [11]:
# Compute new features selected from the fitted model
# If just to read the features use compute_features=False
# Saves features as Loads FTS_NAME_tr_SFM_COLS
SFM_COLUMNS = smf.select_features(cv_new_model=cvloss)
smf.show_no_fts_label(SFM_COLUMNS)

Recomputing current model features from fitted model fts_range_abun_to_temp_XGB_opt_trvl
Computing optimal threshold for each label
basalt - Computing threshold on fts_range_abun_to_temp_XGB_opt_trvl_basalt.joblib.dat
carbonate - Computing threshold on fts_range_abun_to_temp_XGB_opt_trvl_carbonate.joblib.dat
chloride - Computing threshold on fts_range_abun_to_temp_XGB_opt_trvl_chloride.joblib.dat
iron_oxide - Computing threshold on fts_range_abun_to_temp_XGB_opt_trvl_iron_oxide.joblib.dat
oxalate - Computing threshold on fts_range_abun_to_temp_XGB_opt_trvl_oxalate.joblib.dat
oxychlorine - Computing threshold on fts_range_abun_to_temp_XGB_opt_trvl_oxychlorine.joblib.dat
phyllosilicate - Computing threshold on fts_range_abun_to_temp_XGB_opt_trvl_phyllosilicate.joblib.dat
silicate - Computing threshold on fts_range_abun_to_temp_XGB_opt_trvl_silicate.joblib.dat
sulfate - Computing threshold on fts_range_abun_to_temp_XGB_opt_trvl_sulfate.joblib.dat
sulfide - Computing threshold on fts_range

In [12]:
# Retrain the model with newly selected features
TRAIN_FTS_SFM = 'fts_range_abun_to_temp'            # Features selected with SMF() for training.


# ===== TRAIN =====
split_type = 'trvl'

if TRAIN_FTS_SFM:
    # Loads FTS_NAME_tr_SFM_COLS - cols to train with if
    # training is done without full column lenght of input data
    TRAIN_FTS_DICT = smf.load_features(file_name=TRAIN_FTS_SFM)
else: 
    TRAIN_FTS_DICT = None
    
# Train the model- saves features as MODEL_NAME_tr_COLS.txt'
cvloss, submission_model = training.train_tbl(
    df_train=X_trvl,
    df_labels=trvl_labels,
    target_list=target_labels_list,
    df_test=X_vlte,
    split_type=split_type,
    model_algo=MODEL_ALGO,
    sub_name=MODEL_NAME + '_' + split_type,
    fts_select_cols=TRAIN_FTS_DICT,
    )

# Compute validation loss when full model is trained
mloss, mloss_avg = training.compute_valid_loss(submission_model,
                                               valid_files, valid_labels,
                                               target_labels_list,
                                               sub_name=MODEL_NAME + '_' + split_type,
                                               fts_select_cols=TRAIN_FTS_DICT)
print(colored(f'VAL LogLoss: {np.round(mloss_avg, 5)}', 'green'))

Loading feature column names
Reading fts_range_abun_to_temp_XGB_opt_trvl_SFM_COLS.txt
CV training ....
basalt
LogLoss 0.13265015518223017
carbonate
LogLoss 0.1043806934118106
chloride
LogLoss 0.18591879225640234
iron_oxide
LogLoss 0.21104580038718743
oxalate
LogLoss 0.011805043481180515
oxychlorine
LogLoss 0.1554832131014689
phyllosilicate
LogLoss 0.25760361716934876
silicate
LogLoss 0.22854694057843603
sulfate
LogLoss 0.17431480813448336
sulfide
LogLoss 0.08403620244280724
Full training .....
basalt - nfeatures: 228
carbonate - nfeatures: 1583
chloride - nfeatures: 1583
iron_oxide - nfeatures: 1583
oxalate - nfeatures: 11
oxychlorine - nfeatures: 1583
phyllosilicate - nfeatures: 1583
silicate - nfeatures: 1583
sulfate - nfeatures: 243
sulfide - nfeatures: 1583
Saving fts_range_abun_to_temp_XGB_opt_trvl_COLS_sfm.txt
CV LogLoss: 0.15458
VAL LogLoss: 0.01028


**SAM TRAINING**